In [ ]:
import pandas as pd
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", sep=";") 
df

In [ ]:
df.quality.hist(bins=7)

In [ ]:
labels = df.quality # regressziós célváltozó
features = df.iloc[:,:-1] # jellemzőtér

In [ ]:
from sklearn.model_selection import train_test_split
t_features, test_features, t_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=42) # kiértékelő adatbzásis levágása
train_features, valid_features, train_labels, valid_labels = train_test_split(t_features, t_labels, test_size=0.2, random_state=42) # tanító és validációs halmazok leválasztása

In [ ]:
# MSE-t használunk kiértékelési metrikaként
from sklearn.metrics import mean_squared_error

In [ ]:
### döntési fa meta-paraméter hangolással
from sklearn import tree
for d in range(1,20):
  dt = tree.DecisionTreeRegressor(max_depth=d) # döntési fa mélysége
  dt.fit(train_features, train_labels)  # tanítunk a tanító adatbázison
  prediction = dt.predict(valid_features)
  print(d,":",mean_squared_error(prediction, valid_labels))

5 mélységgel legkisebb a hiba

In [ ]:
# az SGD nagyon érzékeny a jellemzők értékkészletének különbségeire (ebben a feladatban a jellemzők különböző skálán mozognak)
# skálázzunk minden jellemzőt ugyanarra az intervallumra
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_train = scaler.fit_transform(train_features)
scaled_valid = scaler.transform(valid_features)

### Lineáris gép meta-paraméter hangolással
from sklearn.linear_model import SGDRegressor
for d in range(-15,3):
  lin = SGDRegressor(alpha=10**d) # regularizációs együttható
  lin.fit(scaled_train, train_labels) 
  prediction = lin.predict(scaled_valid)
  print(d,":",mean_squared_error(prediction, valid_labels))

Nem tud jobban, mint a döntési fa...

In [ ]:
### kNN meta-paraméter hangolással
from sklearn.neighbors import KNeighborsRegressor
for k in range(1,50,2): # páratlan k értékek
  knn = KNeighborsRegressor(n_neighbors=k) # kNN különböző k értékekkel
  knn.fit(scaled_train, train_labels) 
  prediction = knn.predict(scaled_valid)
  print(k,":",mean_squared_error(prediction, valid_labels))

k=15-el a kNN a legjobb (skálázás nélkül is próbáld ki :D )

## Végső kiértékelés a kiértékelő halmazon

In [ ]:
### Teljes címkézett halmazt (train+valid) használhatjuk a végső kiértékeléshez
scaler = StandardScaler()
scaled_train = scaler.fit_transform(t_features)
scaled_test  = scaler.transform(test_features)

In [ ]:
# k=15 kNN a legjobb modellünk:
knn = KNeighborsRegressor(n_neighbors=15)
knn.fit(scaled_train, t_labels)
prediction = knn.predict(scaled_test)
mean_squared_error(prediction, test_labels)

Ne felejtsük el baseline-hoz hasonlítani!

In [ ]:
from sklearn.dummy import DummyRegressor
dummy = DummyRegressor(strategy='mean') # tanító adatbázis címkéinek átlaga lesz mindig a predikció
dummy.fit(scaled_train, t_labels)
mean_squared_error(dummy.predict(scaled_test), test_labels)

A regresszorunk átlagos hibája jóval kisebb, mint a baseline-é!